# Análise exploratória e relatório — Scouts 2015

**Objetivo:** responder às perguntas de correlações, segmentações, outliers e visualização pedidas, usando `pandas`, `numpy`, `matplotlib`, `seaborn`, `scipy` e `sklearn`. O notebook carrega o dataset `/mnt/data/scouts_2015.csv` e gera tabelas, gráficos e uma síntese final.

In [ ]:
# Imports principais
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# Configuracoes de exibicao
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)
plt.rcParams['figure.figsize'] = (10,5)


In [ ]:
# Carregar os dados
path = '/mnt/data/scouts_2015.csv'
df = pd.read_csv(path, low_memory=False)
print('Tamanho do dataset:', df.shape)
df.head()

In [ ]:
# Informação básica e estatísticas descritivas
display(df.info())
display(df.describe(include='all').T)

In [ ]:
# Preparação: identificar colunas numéricas relevantes para análise.
# Ajuste estes nomes caso seu CSV possua colunas com nome diferente.
possible_cols = ['Preço', 'PreçoInicial', 'pontos', 'Pontos', 'Gols', 'G', 'Assistências', 'Assists', 'Assist', 'FC', 'Faltas', 'CA', 'CV', 'RB', 'Roubadas', 'DD', 'DP', 'Defesas', 'Defesa']
cols_lower = [c.lower() for c in df.columns]

# Map common names to columns if present
colmap = {}
for col in df.columns:
    name = col.lower()
    if 'pre' in name and 'ço' in name or 'preco' in name or 'price' in name:
        colmap['price'] = col
    if name in ('pontos','ponto','pontuação','pontuacao') or 'ponto' in name:
        colmap.setdefault('points', col)
    if 'gol' in name:
        colmap.setdefault('goals', col)
    if 'assist' in name:
        colmap.setdefault('assists', col)
    if name in ('fc','faltas','faltas cometidas'):
        colmap.setdefault('fouls', col)
    if name in ('ca','cartao amarelo','cartões amarelos'):
        colmap.setdefault('yellow_cards', col)
    if name in ('cv','cartao vermelho','cartões vermelhos'):
        colmap.setdefault('red_cards', col)
    if 'rb' in name or 'roub' in name:
        colmap.setdefault('steals', col)
    if 'dd' in name or 'dp' in name or 'defes' in name:
        # dd/defesas/defesa - map both
        if 'dd' in name:
            colmap.setdefault('dd', col)
        else:
            colmap.setdefault('saves', col)

colmap

In [ ]:
# Correlação e covariância entre variáveis numéricas
num = df.select_dtypes(include=[np.number])
corr = num.corr()
cov = num.cov()

print('Matriz de correlação (top 10 variáveis numéricas):')
display(corr.round(3).iloc[:10,:10])
print('\nMatriz de covariância (top 10):')
display(cov.round(3).iloc[:10,:10])

# Heatmap de correlação salvo no notebook
plt.figure(figsize=(10,8))
sns.heatmap(corr, annot=False, cmap='viridis')
plt.title('Heatmap de correlação (todas as variáveis numéricas)')
plt.show()

In [ ]:
# Responder perguntas específicas de correlação -- ajustar nomes conforme colmap detectado
price_col = colmap.get('price', None)
points_col = colmap.get('points', None)
goals_col = colmap.get('goals', None)
assists_col = colmap.get('assists', None)
fouls_col = colmap.get('fouls', None)
yellow_col = colmap.get('yellow_cards', None)
red_col = colmap.get('red_cards', None)
steals_col = colmap.get('steals', None)

print('Mapeamento de colunas detectado:', colmap)

def spearman_and_pearson(a,b):
    valid = df[[a,b]].dropna()
    pearson = stats.pearsonr(valid[a], valid[b]) if len(valid)>1 else (np.nan,np.nan)
    spearman = stats.spearmanr(valid[a], valid[b]) if len(valid)>1 else (np.nan,np.nan)
    return {'pearson': pearson, 'spearman': spearman, 'n': len(valid)}

if price_col and points_col:
    print('\nCorrelação preço x pontos:')
    print(spearman_and_pearson(price_col, points_col))

if goals_col and points_col:
    print('\nCorrelação gols x pontos:')
    print(spearman_and_pearson(goals_col, points_col))

if goals_col and assists_col:
    print('\nCorrelação gols x assistências:')
    print(spearman_and_pearson(goals_col, assists_col))

if fouls_col and (yellow_col or red_col):
    print('\nCorrelação faltas x cartões:')
    if yellow_col:
        print('Faltas x Cartões Amarelos:', spearman_and_pearson(fouls_col, yellow_col))
    if red_col:
        print('Faltas x Cartões Vermelhos:', spearman_and_pearson(fouls_col, red_col))

In [ ]:
# Segmentações: média de pontos por clube, contribuição por posição em RB, rodada com mais gols
# Detect possible column names for club, position, rodada
club_col = None
position_col = None
round_col = None
for c in df.columns:
    lc = c.lower()
    if 'club' in lc or 'clube' in lc or 'time' in lc:
        club_col = c if club_col is None else club_col
    if 'pos' in lc and 'posição' in lc or 'posição' in lc or 'posicao' in lc or lc=='pos':
        position_col = c if position_col is None else position_col
    if 'rodada' in lc or 'round' in lc:
        round_col = c if round_col is None else round_col

print('club_col, position_col, round_col =', club_col, position_col, round_col)

# média de pontos por jogador por clube (se existir coluna de clube)
if club_col and points_col:
    club_points = df.groupby(club_col)[points_col].mean().sort_values(ascending=False)
    display(club_points.head(10))
else:
    print('Coluna de clube ou pontos não detectada automaticamente. Ajuste nomes no notebook.')

# posições que mais contribuíram em roubadas de bola (RB)
if position_col and steals_col:
    pos_rb = df.groupby(position_col)[steals_col].sum().sort_values(ascending=False)
    display(pos_rb)
else:
    print('Coluna de posição ou RB não detectada automaticamente. Ajuste nomes no notebook.')

# rodada mais produtiva em gols
if round_col and goals_col:
    round_goals = df.groupby(round_col)[goals_col].sum().sort_values(ascending=False)
    display(round_goals.head(10))
else:
    print('Coluna de rodada ou gols não detectada automaticamente. Ajuste nomes no notebook.')

In [ ]:
# Outliers: z-score, IQR, boxplots para pontuação, preço e defesas (se existirem)
def detect_outliers_zscore(series, thresh=3.0):
    s = series.dropna()
    z = np.abs(stats.zscore(s))
    return s.index[z>thresh].tolist(), z

outlier_report = {}

for col in [points_col, price_col, 'DD' if 'DD' in df.columns else None]:
    if col and col in df.columns:
        idxs, z = detect_outliers_zscore(df[col])
        outlier_report[col] = {'n_outliers_zscore': len(idxs), 'sample_outliers_index': idxs[:10]}

outlier_report

In [ ]:
# Boxplot: pontuação por posição (se houver position_col e points_col)
if position_col and points_col:
    plt.figure(figsize=(12,6))
    sns.boxplot(data=df, x=position_col, y=points_col)
    plt.title('Distribuição de pontos por posição')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print('Não foi possível gerar boxplot por posição; ajuste nomes de colunas.')

In [ ]:
# Scatter: gols x assistências, destacando clubes mais produtivos (se existir)
if goals_col and assists_col and club_col:
    top_clubs = df.groupby(club_col)[goals_col].sum().sort_values(ascending=False).head(5).index.tolist()
    plt.figure(figsize=(10,6))
    sns.scatterplot(data=df, x=goals_col, y=assists_col, hue=club_col, alpha=0.7)
    plt.title('Gols x Assistências (por jogador)')
    plt.show()
    print('Top clubes por gols:', top_clubs)
else:
    print('Não foi possível gerar scatter gols x assistências; ajuste nomes de colunas.')

In [ ]:
# PCA para entender padrões de variância entre features numéricas (ex.: pontos, gols, assists, roubadas)
num_features = ['pontos','gols','assists','rb','fc','dd','dp']
# map to detected columns where possible
feature_map = {}
for f in num_features:
    for col in df.columns:
        if f in col.lower():
            feature_map[f] = col
# keep only those found
features = [v for v in feature_map.values() if v in df.columns]
print('Features usadas no PCA detectadas automaticamente:', features)

if len(features) >= 2:
    X = df[features].dropna()
    scaler = StandardScaler()
    Xs = scaler.fit_transform(X)
    pca = PCA(n_components=min(4, Xs.shape[1]))
    pcs = pca.fit_transform(Xs)
    print('Explained variance ratio:', pca.explained_variance_ratio_)
    # Scree plot
    plt.figure()
    plt.plot(np.arange(1, len(pca.explained_variance_ratio_)+1), pca.explained_variance_ratio_, marker='o')
    plt.xlabel('Componente principal')
    plt.ylabel('Variância explicada')
    plt.title('Scree plot — PCA')
    plt.show()
else:
    print('Poucas features numéricas detectadas automaticamente para PCA. Ajuste manualmente no notebook.')

In [ ]:
# Síntese automática (modelo): gera um arquivo de relatório curto baseado nos resultados detectados
report_lines = []
report_lines.append('# Relatório automático — Scouts 2015\n')
report_lines.append('Este relatório foi gerado automaticamente pelo notebook. Ajuste e refine as análises conforme necessário.\\n')\

# Add small synopsis based on what we computed (best-effort)
if price_col and points_col and price_col in df.columns and points_col in df.columns:
    corr_p = df[[price_col, points_col]].dropna().corr().iloc[0,1]
    report_lines.append(f'- Correlação preço x pontos (Pearson aprox.): {corr_p:.3f}\\n')
else:
    report_lines.append('- Correlação preço x pontos: coluna não detectada automaticamente.\\n')

if goals_col and assists_col and goals_col in df.columns and assists_col in df.columns:
    corr_ga = df[[goals_col, assists_col]].dropna().corr().iloc[0,1]
    report_lines.append(f'- Correlação gols x assistências (Pearson aprox.): {corr_ga:.3f}\\n')
else:
    report_lines.append('- Correlação gols x assistências: coluna não detectada automaticamente.\\n')

report_lines.append('\\n## Observações sobre outliers\\n')
report_lines.append('Verifique boxplots e resultados do z-score/IQR no notebook para detectar performances extremas.\\n')

report_text = '\\n'.join(report_lines)
with open('/mnt/data/scouts_report.md', 'w', encoding='utf-8') as f:
    f.write(report_text)

print('Relatório resumido salvo em /mnt/data/scouts_report.md')